In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
home_sales_df.show()


In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4_bed_df = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
""")
avg_price_4_bed_df.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3bed_3bath_df = spark.sql("""
    SELECT YEAR(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year_built
    ORDER BY year_built
""")
avg_price_3bed_3bath_df.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_filtered_df = spark.sql("""
    SELECT YEAR(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY year_built
    ORDER BY year_built
""")
avg_price_filtered_df.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()
avg_price_per_view_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
avg_price_per_view_df.show()
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

In [ ]:
# 8. Check if the table is cached.
print("Is the table cached?", spark.catalog.isCached("home_sales"))

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_price_per_view_cached_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
avg_price_per_view_cached_df.show()
print("--- %s seconds (cached) ---" % (time.time() - start_time))

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet", mode="overwrite")

In [ ]:
# 11. Read the formatted parquet data.
partitioned_df = spark.read.parquet("home_sales_partitioned.parquet")
partitioned_df.show()

In [ ]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales_partitioned")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
avg_price_per_view_partitioned_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_partitioned
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")
avg_price_per_view_partitioned_df.show()
print("--- %s seconds (partitioned) ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
print("Is the table cached after uncaching?", spark.catalog.isCached("home_sales"))
